In [85]:
import os
import numpy as np
import pandas as pd
import mne
import scipy.io
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import FastICA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score



In [86]:
#filePath= '/Users/yaman/Library/CloudStorage/GoogleDrive-s203192@stn.nagaokaut.ac.jp/マイドライブ/実験データ/20230519_zikken'
#filePath= '/Users/yaman/Library/CloudStorage/GoogleDrive-s203192@stn.nagaokaut.ac.jp/マイドライブ/実験データ/20230529_1_zikken'
filePath= '/Users/yaman/Library/CloudStorage/GoogleDrive-s203192@stn.nagaokaut.ac.jp/マイドライブ/実験データ/20230214_zikken'

brainAdress = ['Fp1','AF7','AF3','F1','F3','F5','F7','FT7','FC5',
'FC3','FC1','C1','C3','C5','T7','TP7','CP5','CP3','CP1','P1','P3',
'P5','P7','P9','PO7','PO3','O1','Iz','Oz','POz','Pz','CPz','Fpz','Fp2',
'AF8','AF4','AFz','Fz','F2','F4','F6','F8','FT8','FC6','FC4','FC2',
'FCz','Cz','C2','C4','C6','T8','TP8','CP6','CP4','CP2','P2','P4',
'P6','P8','P10','PO8','PO4','O2']

# dataクラス
class O_Data:
    def __init__(self, eeg, index, label):
        self.eeg = eeg
        self.index = index
        self.label = label
        self.stack = []

    def push(self, item):
        self.stack.append(item)

o_data65_label1 = list()#label1のデータ
o_data = list()
label_data = list()#labelを結合したデータ


All_data = list()  # 全データを結合

def Standardization(data): #標準化
    after_data = scipy.stats.zscore(data)
    #b = np.average(after_data)
    #c = np.var(after_data)
    return after_data

file_name = list()  # すべての.matファイルの名前
for file in os.listdir(filePath):
    base, ext = os.path.splitext(file)
    if ext == '.mat':
        file_name = sorted(file_name)
        file_name.append(file)
print('Filelist')
print(file_name)

Filelist
['EEG_grasping_14-Feb-2023_sub1-1.mat', 'EEG_grasping_14-Feb-2023_sub1-2.mat', 'EEG_grasping_14-Feb-2023_sub1-3.mat', 'EEG_grasping_14-Feb-2023_sub2-2.mat', 'EEG_grasping_14-Feb-2023_sub2-3.mat', 'EEG_grasping_14-Feb-2023_sub3-1.mat', 'EEG_grasping_14-Feb-2023_sub3-2.mat', 'EEG_grasping_14-Feb-2023_sub3-3.mat', 'EEG_grasping_14-Feb-2023_sub4-1.mat', 'EEG_grasping_14-Feb-2023_sub4-2.mat', 'EEG_grasping_14-Feb-2023_sub4-3.mat', 'EEG_grasping_14-Feb-2023_sub5-1.mat', 'EEG_grasping_14-Feb-2023_sub5-2.mat', 'EEG_grasping_14-Feb-2023_sub5-3.mat', 'EEG_grasping_14-Feb-2023_sub2-1.mat']


In [87]:
#解析するデータを選択
file_name1 = [file_name[3],file_name[4],file_name[5],file_name[6],file_name[7],file_name[14]]
file_num1 = len(file_name1)
trial_num = 30
all_trial = len(file_name1) * trial_num
all_d = 0
CH = 64
#label1を読み込む
for s in range(file_num1):
    Dictionary = scipy.io.loadmat(filePath+"/"+file_name1[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:66, :]
                index = b[1]
                samplerate = b[2]
                label = b[5][0, 0]
                o_data = O_Data(eeg, index, label)
                o_data65_label1.append(o_data)
print("image data_loading")

image data_loading


In [88]:
time_data =8*1024

In [89]:
c1 = o_data65_label1[0].eeg[: , 0:time_data]
for n in range(len(file_name1)*trial_num-1):  # all_trial
    g1 = o_data65_label1[n+1].eeg[: , 0:time_data]
    c1 = np.dstack([c1, g1])


In [90]:
all_data = c1.transpose(2,0,1)
print(np.shape(all_data))
all_data = all_data.astype(np.double)

(180, 66, 8192)


In [91]:
filter_bandpass = signal.firwin(numtaps=51,cutoff=[8,32],fs = 1024,pass_zero=False)

In [92]:
for n in range(3):
    for j in range(60):
        for k in range(66):
            #all_data[n*60+j][k]= abs(all_data[n*60+j][k]-np.mean(all_data[n*60+j][0:66],axis=0))
            all_data[n*60+j][k] = signal.lfilter(filter_bandpass,1,all_data[n*60+j][k])
        #all_data[n*60+j] = scipy.stats.zscore(all_data[n*60+j])
    #all_data[60*n:60*n+60] = scipy.stats.zscore(all_data[60*n:60*n+60])

In [93]:
# plt.plot(all_data[2][5][1024*6:1024*10])
# plt.show()

In [94]:
label_data = list()#labelを結合したデータ
for n in range(all_trial):
    label_data.append(o_data65_label1[n].label)
print(label_data)
print(np.shape(label_data))

[2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1]
(180,)


In [95]:
#numpy配列の定義
task1 = o_data65_label1[0].eeg[: , 0:time_data]
task2 = o_data65_label1[0].eeg[: , 0:time_data]
for n in range(all_trial):
    if label_data[n] == 1:
        join = o_data65_label1[n].eeg[: , 0:time_data]
        task1 = np.dstack([task1, join])
    if label_data[n] == 2:
        join = o_data65_label1[n].eeg[: , 0:time_data]
        task2 = np.dstack([task2, join])
#task1低速度
task1_data = task1.transpose(2,0,1)
task1_data = task1_data.astype(np.double)
task1_data = np.delete(task1_data,0,axis=0)
print(np.shape(task1_data))
#task2低速度
task2_data = task2.transpose(2,0,1)
task2_data = task2_data.astype(np.double)
task2_data = np.delete(task2_data,0,axis=0)
print(np.shape(task2_data))

(90, 66, 8192)
(90, 66, 8192)


In [96]:
processed_task1 = task1_data[:,range(64),1024*2:time_data]
processed_task2 = task2_data[:,range(64),1024*2:time_data]


In [97]:
class_data = np.concatenate([processed_task1,processed_task2])
class_label = np.zeros(180,dtype=int)
class_label[90:] = 1

In [98]:
X_train, X_test, Y_train, Y_test = train_test_split(class_data, class_label, test_size=0.7)

In [99]:
# Assemble a classifier
lda = LinearDiscriminantAnalysis()
#csp = CSP(n_components=4, reg=None, norm_trace=False, transform_into='average_power',cov_est='epoch')
#csp = CSP(n_components=4, reg=None, norm_trace=False, transform_into='csp_space',cov_est='epoch')

csp = CSP(n_components=4, reg=None, norm_trace=False, transform_into='average_power',cov_est='epoch')
# Use scikit-learn Pipeline with cross_val_score function
n_channels = 64
sampling_freq = 1024  # in Hertz
info = mne.create_info(ch_names=brainAdress, ch_types="eeg", sfreq=sampling_freq)
info.set_montage('biosemi64')

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, ...
 chs: 64 EEG
 custom_ref_applied: False
 dig: 67 items (3 Cardinal, 64 EEG)
 highpass: 0.0 Hz
 lowpass: 512.0 Hz
 meas_date: unspecified
 nchan: 64
 projs: []
 sfreq: 1024.0 Hz
>

In [100]:
# Printing the results
class_balance = np.mean(class_label == class_label[0])
class_balance = max(class_balance, 1. - class_balance)
#print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),class_balance))

# plot CSP patterns estimated on full data for visualization
for n in range(6):
    CSP_bandpass = signal.firwin(numtaps=51,cutoff=[n*4+8,n*4+12],fs = 1024,pass_zero=False)
    filtered_csp_train = signal.lfilter(CSP_bandpass,1,X_train)
    filtered_csp_test = signal.lfilter(CSP_bandpass,1,X_test)
    tmp_train = csp.fit_transform(filtered_csp_train, Y_train)
    tmp_test = csp.transform(filtered_csp_test)
    if n==0:
        features_train = tmp_train
        features_test = tmp_test
    else:
        features_train=np.concatenate((features_train,tmp_train),axis=1)
        features_test=np.concatenate((features_test,tmp_test),axis=1)
print(features_train.shape)
print(features_test.shape)

Computing rank from data with rank=None
    Using tolerance 1.7e+05 (2.2e-16 eps * 64 dim * 1.2e+19  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.7e+05 (2.2e-16 eps * 64 dim * 1.2e+19  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.7e+05 (2.2e-16 eps * 64 dim * 1.2e+19  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.7e+05 (2.2e-16 eps * 64 dim * 1.2e+19  max singular value)


In [101]:
#class_data_cspfit2 = np.reshape(features_test,(126,24*6144))

X_train2, X_test2, Y_train2, Y_test2 = train_test_split(features_test,Y_test, test_size=0.2,random_state=42)

In [102]:
model_gs_cv = MLPClassifier()
random_search = {'batch_size': [10, 20, 30, 130],
               'hidden_layer_sizes': [(150, 150), (100, 100, 50), (150, 100, 50)],
               'max_iter': [400, 500, 600, 700, 800],
               'random_state': [0]}
gscv = GridSearchCV(model_gs_cv, random_search, cv=3, verbose=2)
gscv.fit(X_train2, Y_train2)


Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] END batch_size=10, hidden_layer_sizes=(150, 150), max_iter=400, random_state=0; total time=   0.3s
[CV] END batch_size=10, hidden_layer_sizes=(150, 150), max_iter=400, random_state=0; total time=   0.5s
[CV] END batch_size=10, hidden_layer_sizes=(150, 150), max_iter=400, random_state=0; total time=   0.5s
[CV] END batch_size=10, hidden_layer_sizes=(150, 150), max_iter=500, random_state=0; total time=   0.4s
[CV] END batch_size=10, hidden_layer_sizes=(150, 150), max_iter=500, random_state=0; total time=   0.5s
[CV] END batch_size=10, hidden_layer_sizes=(150, 150), max_iter=500, random_state=0; total time=   0.4s
[CV] END batch_size=10, hidden_layer_sizes=(150, 150), max_iter=600, random_state=0; total time=   0.4s
[CV] END batch_size=10, hidden_layer_sizes=(150, 150), max_iter=600, random_state=0; total time=   0.4s
[CV] END batch_size=10, hidden_layer_sizes=(150, 150), max_iter=600, random_state=0; total time=   0.4s
[C

/Users/yaman/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:621: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(


[CV] END batch_size=130, hidden_layer_sizes=(150, 150), max_iter=400, random_state=0; total time=   0.3s


/Users/yaman/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:621: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(


[CV] END batch_size=130, hidden_layer_sizes=(150, 150), max_iter=400, random_state=0; total time=   0.2s


...

[CV] END batch_size=130, hidden_layer_sizes=(150, 100, 50), max_iter=800, random_state=0; total time=   0.2s
[CV] END batch_size=130, hidden_layer_sizes=(150, 100, 50), max_iter=800, random_state=0; total time=   0.2s


/Users/yaman/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:621: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/Users/yaman/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:621: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(


[CV] END batch_size=130, hidden_layer_sizes=(150, 100, 50), max_iter=800, random_state=0; total time=   0.2s


/Users/yaman/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:621: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(


GridSearchCV(cv=3, estimator=MLPClassifier(),
             param_grid={'batch_size': [10, 20, 30, 130],
                         'hidden_layer_sizes': [(150, 150), (100, 100, 50),
                                                (150, 100, 50)],
                         'max_iter': [400, 500, 600, 700, 800],
                         'random_state': [0]},
             verbose=2)

In [103]:
#正答率が最高だった時とその時のパラメータを取得
bestScore = gscv.best_score_
bestParam = gscv.best_params_

#表示
print(f'{bestParam}のときに正答率が最高で{bestScore}')

{'batch_size': 130, 'hidden_layer_sizes': (100, 100, 50), 'max_iter': 400, 'random_state': 0}のときに正答率が最高で0.459298871063577


In [104]:
# 正答率が最高のモデルを取得
best = gscv.best_estimator_

#↑で予測し、その正答率を表示
best.score(X_test2, Y_test2)

0.5

In [105]:
results = []
for hidden_layer_sizes in [10, 100, 1000]:
    for solver in ['sgd', 'adam', 'lbfgs']:
        for activation in ['identity', 'logistic', 'tanh', 'relu']:
            for learning_rate_init in [0.1, 0.01, 0.001]:
                clf = MLPClassifier(max_iter=10000,
                       hidden_layer_sizes=(hidden_layer_sizes,), 
                       activation=activation, solver=solver,
                       learning_rate_init=learning_rate_init)
                clf.fit(X_train2, Y_train2)
                score = clf.score(X_test2, Y_test2)
                results.append([hidden_layer_sizes, activation,
                       solver, learning_rate_init, score])

grid_search 

In [106]:
res = pd.DataFrame([dat for dat in sorted(results, key=lambda f: f[4],
            reverse=True)], columns=["hidden_layer_sizes", 
            "activation", "solver", "learning_rate_init", "score"])

In [107]:
display(res)
# CSV ファイル (employee.csv) として出力
res.to_csv("result.csv")

,hidden_layer_sizes,activation,solver,learning_rate_init,score
0,10,tanh,adam,0.001,0.653846
1,10,logistic,lbfgs,0.001,0.653846
2,10,relu,lbfgs,0.010,0.615385
3,100,tanh,adam,0.100,0.615385
4,10,relu,sgd,0.001,0.576923
...,...,...,...,...,...
103,100,logistic,sgd,0.010,0.384615
104,100,logistic,adam,0.010,0.384615
105,1000,identity,sgd,0.010,0.384615
106,1000,logistic,sgd,0.010,0.384615


In [108]:
model_gs_cv2 = MLPClassifier()
random_search2 = {'activation' :['relu'], 
                  'learning_rate_init':[0.001],
                'hidden_layer_sizes': [10],
               'max_iter': [10000],
               'solver':['lbfgs'],
               'random_state': [42]}
gscv2 = GridSearchCV(model_gs_cv2, random_search2, cv=3, verbose=3)
gscv2.fit(X_train2, Y_train2)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END activation=relu, hidden_layer_sizes=10, learning_rate_init=0.001, max_iter=10000, random_state=42, solver=lbfgs;, score=0.529 total time=   0.0s
[CV 2/3] END activation=relu, hidden_layer_sizes=10, learning_rate_init=0.001, max_iter=10000, random_state=42, solver=lbfgs;, score=0.303 total time=   0.0s
[CV 3/3] END activation=relu, hidden_layer_sizes=10, learning_rate_init=0.001, max_iter=10000, random_state=42, solver=lbfgs;, score=0.455 total time=   0.0s


GridSearchCV(cv=3, estimator=MLPClassifier(),
             param_grid={'activation': ['relu'], 'hidden_layer_sizes': [10],
                         'learning_rate_init': [0.001], 'max_iter': [10000],
                         'random_state': [42], 'solver': ['lbfgs']},
             verbose=3)

In [109]:
#正答率が最高だった時とその時のパラメータを取得
bestScore = gscv2.best_score_
bestParam = gscv2.best_params_

#表示
print(f'{bestParam}のときに正答率が最高で{bestScore}')

{'activation': 'relu', 'hidden_layer_sizes': 10, 'learning_rate_init': 0.001, 'max_iter': 10000, 'random_state': 42, 'solver': 'lbfgs'}のときに正答率が最高で0.4289958407605467


In [110]:
# 正答率が最高のモデルを取得
best = gscv2.best_estimator_

#↑で予測し、その正答率を表示
best.score(X_test2, Y_test2)

0.6153846153846154

In [111]:
# from sklearn.model_selection import KFold
# kf = KFold(n_splits=5)

In [112]:
# class_data_cspfit2 = np.reshape(features_test,(126,24))

X_train3, X_test3, Y_train3, Y_test3 = train_test_split(features_test,Y_test, test_size=0.2,random_state=42)

In [113]:
model_svm =SVC(kernel='linear')
model_svm.fit(X_train3,Y_train3)

# SVC(C=1.0, class_weight=None, coef0=0.0,
#   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#   max_iter=-1, probability=False, random_state=None, shrinking=True,
#   tol=0.001, verbose=False)

print(model_svm.score(X_train3, Y_train3))
print(model_svm.score(X_test3, Y_test3))

0.66
0.5769230769230769


In [114]:
params = {
    "C":np.arange(0.1,1,0.005),
    "kernel":["linear", "poly", "rbf", "sigmoid"],
    "gamma":np.arange(0.0001,0.1,0.05)
}
grid = GridSearchCV(model_svm, params,scoring="accuracy", cv=5)

grid.fit(X_train3, Y_train3)
print(grid.best_score_)
print(grid.best_params_)

pred = grid.predict(X_test3)
print(classification_report(Y_test3, pred))

0.55
{'C': 0.6900000000000005, 'gamma': 0.050100000000000006, 'kernel': 'poly'}
              precision    recall  f1-score   support

           0       0.46      0.55      0.50        11
           1       0.62      0.53      0.57        15

    accuracy                           0.54        26
   macro avg       0.54      0.54      0.54        26
weighted avg       0.55      0.54      0.54        26

